In [3]:
import torch.nn as nn

In [6]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [7]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [8]:
model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes=257)

In [1]:
path = "./data/256_ObjectCategoriesSplit/"

In [3]:
from torchvision import datasets
from torchvision import transforms

In [4]:
data_transforms = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [6]:
train = datasets.ImageFolder(path+"train", transform=data_transforms)
valid = datasets.ImageFolder(path+"valid", transform=data_transforms)

In [7]:
import torch 
train_x = [x[0] for x in train]
len(train_x)

24385

In [8]:
type(train_x)

list

In [9]:
import numpy as np
train_x = torch.stack(train_x)
type(train_x)


torch.Tensor

In [10]:
train_x.shape

torch.Size([24385, 3, 224, 224])

In [11]:
train_x[0]

tensor([[[ 1.0331,  1.0502,  0.9988,  ...,  1.2899,  1.4269,  1.4440],
         [ 0.8789,  0.9646,  0.9817,  ...,  1.4440,  1.4783,  1.4783],
         [ 0.8104,  0.9474,  1.0331,  ...,  1.4783,  1.5125,  1.5125],
         ...,
         [ 0.8447,  0.8789,  1.1015,  ...,  1.0159,  0.9303,  1.0673],
         [ 0.7933,  0.7933,  1.0502,  ...,  1.0159,  1.0502,  1.1358],
         [ 0.7591,  0.7591,  0.9988,  ...,  1.2385,  1.1187,  1.1529]],

        [[ 1.5007,  1.5182,  1.4657,  ...,  1.7808,  1.9209,  1.9384],
         [ 1.3431,  1.4307,  1.4482,  ...,  1.9384,  1.9734,  1.9734],
         [ 1.2731,  1.4132,  1.5007,  ...,  1.9734,  2.0084,  2.0084],
         ...,
         [ 1.2381,  1.2731,  1.5007,  ...,  1.3782,  1.2906,  1.4307],
         [ 1.1856,  1.1856,  1.4482,  ...,  1.3782,  1.4132,  1.5007],
         [ 1.1506,  1.1506,  1.3957,  ...,  1.6057,  1.4832,  1.5182]],

        [[ 1.7163,  1.7337,  1.6814,  ...,  2.0648,  2.2043,  2.2217],
         [ 1.5594,  1.6465,  1.6640,  ...,  2

In [12]:
train_y = [x[1] for x in train]
len(train_y)

24385

In [13]:
type(train_y[0])

int

In [14]:
train_y[0]

0

In [17]:
train_y = torch.IntTensor(train_y)
type(train_y)

torch.Tensor

In [18]:
train_y

tensor([  0,   0,   0,  ..., 256, 256, 256], dtype=torch.int32)

In [1]:
from fastai import *
from fastai.vision import *

In [2]:
doc(ConvLearner)

In [10]:
doc(get_transforms)

In [11]:
doc(ImageDataBunch)